In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

    

import torchvision
import torchvision.transforms as transforms

from models import *   # bring everything in the folder models

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
batch_size = 128
model_name = "VGG16_quant_2bit1"
model = VGG16_quant()
print(model)


normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
# This cell won't be given, but students will complete the training

lr = 0.05
weight_decay = 1e-4
epochs = 250
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.615 (0.615)	Data 0.114 (0.114)	Loss 2.4758 (2.4758)	Prec 10.156% (10.156%)
Epoch: [0][100/391]	Time 0.044 (0.051)	Data 0.001 (0.003)	Loss 2.4063 (3.2931)	Prec 7.812% (10.435%)
Epoch: [0][200/391]	Time 0.047 (0.048)	Data 0.001 (0.002)	Loss 2.4781 (2.8279)	Prec 12.500% (10.801%)
Epoch: [0][300/391]	Time 0.045 (0.047)	Data 0.002 (0.002)	Loss 2.3102 (2.6678)	Prec 10.938% (11.078%)
Validation starts
Test: [0/79]	Time 0.116 (0.116)	Loss 2.2623 (2.2623)	Prec 14.844% (14.844%)
 * Prec 11.370% 
best acc: 11.370000
Epoch: [1][0/391]	Time 0.157 (0.157)	Data 0.122 (0.122)	Loss 2.3877 (2.3877)	Prec 10.938% (10.938%)
Epoch: [1][100/391]	Time 0.043 (0.048)	Data 0.001 (0.005)	Loss 2.3085 (2.3142)	Prec 7.812% (11.997%)
Epoch: [1][200/391]	Time 0.044 (0.046)	Data 0.001 (0.003)	Loss 2.3162 (2.3094)	Prec 15.625% (12.150%)
Epoch: [1][300/391]	Time 0.045 (0.046)	Data 0.006 (0.003)	Loss 2.2923 (2.3038)	Prec 11.719% (12.448%)
Validation starts
Test: [0/79]	Time 0.116 (0.116)	Loss 2.27

KeyboardInterrupt: 

In [4]:
PATH = "result/VGG16_quant_2bit1/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 7896/10000 (79%)



In [5]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []

save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.register_forward_pre_hook(save_output)

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)


In [6]:
#print(model.layer1[0].conv1)

w_bit = 4
weight_q = model.features[27].weight_q # quantized value is stored during the training
w_alpha = model.features[27].weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here
w_delta = w_alpha/(2**(w_bit-1)-1)    # delta can be calculated by using alpha and w_bit
weight_int = weight_q/w_delta # w_int can be calculated by weight_q and w_delta
#print(weight_int) # you should see clean integer numbers
#weight_int.size()

In [7]:
x_bit = 2
x = save_output.outputs[8][0]  # input of the 2nd conv layer
x_alpha  = model.features[27].act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
#print(x_int) # you should see clean integer numbers 

In [8]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=16, kernel_size = 3, padding = 1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int = conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int*w_delta*x_delta  # recover with x_delta and w_delta
#print(output_recovered)
l1 = nn.ReLU()
output_recovered = l1(output_recovered)

output_ref = save_output.outputs[9][0]

difference = abs(output_ref - output_recovered)
print(difference.mean())

tensor(1.5333e-06, device='cuda:0', grad_fn=<MeanBackward0>)


In [9]:
a_int = x_int[0,:,:,:]  # pick only one input out of batch
#print(a_int.size())

w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
#print(w_int.size())

padding = 1
stride = 1
array_size_oc = 8 # row and column number
array_size_ic = 16 # row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(int(len(icg)/array_size_ic))
oc_tileg = range(int(len(ocg)/array_size_oc))

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
#print(a_pad.size()) #[8, , ]

a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
#print(a_pad.size())

a_tile = torch.zeros(len(ic_tileg), array_size_ic,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), array_size_oc, array_size_ic, len(kijg)).cuda() 
print(a_tile.size())
print(w_tile.size())

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*array_size_ic:(ic_tile+1)*array_size_ic,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[oc_tile*len(oc_tileg) + ic_tile,:,:,:] = w_int[oc_tile*array_size_oc:(oc_tile+1)*array_size_oc, ic_tile*array_size_ic:(ic_tile+1)*array_size_ic, :]
print(a_tile.size())
print(w_tile.size())


###########################################

p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size_oc, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for ic_tile in ic_tileg:       # Tiling into array_sizeXarray_size array
        for oc_tile in oc_tileg:   # Tiling into array_sizeXarray_size array        
            for nij in p_nijg:       # time domain, sequentially given input
                    m = nn.Linear(array_size_ic, array_size_oc, bias=False)
                    #m.weight = torch.nn.Parameter(w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, kij])
                    m.weight = torch.nn.Parameter(w_tile[len(oc_tileg)*oc_tile+ic_tile,:,:,kij])
                    psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,:,nij]).cuda()


torch.Size([1, 16, 36])
torch.Size([1, 8, 16, 9])
torch.Size([1, 16, 36])
torch.Size([1, 8, 16, 9])


In [29]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1)
o_nijg = range(o_ni_dim**2)    
print(a_pad_ni_dim)
print(o_ni_dim)
    
out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:
        for ic_tile in ic_tileg:    
            for oc_tile in oc_tileg:   
                out[oc_tile*array_size_oc:(oc_tile+1)*array_size_oc, o_nij] = out[oc_tile*array_size_oc:(oc_tile+1)*array_size_oc, o_nij] + \
                psum[ic_tile, oc_tile, :, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]

print(out.size())
print(psum.size())

6
4
0
1
2
6
7
8
12
13
14
1
2
3
7
8
9
13
14
15
2
3
4
8
9
10
14
15
16
3
4
5
9
10
11
15
16
17
6
7
8
12
13
14
18
19
20
7
8
9
13
14
15
19
20
21
8
9
10
14
15
16
20
21
22
9
10
11
15
16
17
21
22
23
12
13
14
18
19
20
24
25
26
13
14
15
19
20
21
25
26
27
14
15
16
20
21
22
26
27
28
15
16
17
21
22
23
27
28
29
18
19
20
24
25
26
30
31
32
19
20
21
25
26
27
31
32
33
20
21
22
26
27
28
32
33
34
21
22
23
27
28
29
33
34
35
torch.Size([8, 16])
torch.Size([1, 1, 8, 36, 9])


In [11]:
out_2D = torch.reshape(out, (out.size(0), o_ni_dim, -1))

difference = (out_2D - output_int[0,:,:,:])
print(difference.sum())

tensor(1.2398e-05, device='cuda:0', grad_fn=<SumBackward0>)


In [20]:
### show this cell partially. The following cells should be printed by students ###
tile_id = 0 
#nij = 36 # just a random number
X = a_tile[tile_id,:,:]  # [tile_num, array row num, time_steps]
print(X.size())

bit_precision = 2
file = open('activation.txt', 'w') #write to file
file.write('#time0row15[msb-lsb],time0row14[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row15[msb-lsb],time1row14[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:02b}'.format(round(X[15-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    
#print(X)

torch.Size([16, 36])


In [16]:
tile_id = 0 
kij = 0
W = w_tile[tile_id,:,:,kij]  # w_tile[tile_num, array col num, array row num, kij]

bit_precision = 4
print(w_tile.size())

for kij in range(w_tile.size(3)):
    W = w_tile[tile_id,:,:,kij]
    file = open('weight_kij'+str(kij)+'_reference.txt', 'w') #write to file
    file.write('#col0row15[msb-lsb],col0row14[msb-lst],....,col0row0[msb-lst]#\n')
    file.write('#col1row15[msb-lsb],col1row14[msb-lst],....,col1row0[msb-lst]#\n')
    file.write('#................#\n')
    for i in range(W.size(0)):
        for j in range(W.size(1)):
            temp = round(W[i,15-j].item())
            if temp<0:
                temp=temp+16
            W_bin = '{0:04b}'.format(temp)
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file 



torch.Size([1, 8, 16, 9])


In [18]:
tile_id = 0 
kij = 0
W = w_tile[tile_id,:,:,kij]  # w_tile[tile_num, array col num, array row num, kij]

bit_precision = 4
print(w_tile.size())

for kij in range(w_tile.size(3)):
    W = w_tile[tile_id, :, :, kij]
    file = open('weight_kij'+str(kij)+'.txt', 'w') #write to file
    file.write('#col0row14[msb-lsb],col0row12[msb-lst],....,col0row0[msb-lst]#\n')
    file.write('#col0row15[msb-lsb],col0row13[msb-lst],....,col0row1[msb-lst]#\n')
    file.write('#................#\n')
    for i in range(W.size(0)):
        #Even
        for j in range(W.size(1)):
            if j % 2 != 0:
                temp = round(W[i, 15-j].item())
                if temp < 0:
                    temp = temp + 2**bit_precision
                W_bin = '{:04b}'.format(temp)
                for k in range(bit_precision):
                    file.write(W_bin[k])  
        file.write('\n')
        #Odd
        for j in range(W.size(1)):
            if j % 2 == 0:
                temp = round(W[i, 15-j].item())
                if temp < 0:
                    temp = temp + 2**bit_precision
                W_bin = '{:04b}'.format(temp)
                for k in range(bit_precision):
                    file.write(W_bin[k])  
        file.write('\n')

torch.Size([1, 8, 16, 9])


In [19]:
W[:,:] # check this number with your 2nd line in weight.txt

tensor([[ 5.0000, -7.0000,  7.0000,  3.0000,  0.0000,  0.0000,  0.0000, -1.0000,
          4.0000,  2.0000,  2.0000,  0.0000,  1.0000, -2.0000,  4.0000,  0.0000],
        [ 0.0000, -1.0000, -4.0000, -0.0000,  6.0000, -2.0000, -0.0000, -0.0000,
         -1.0000,  2.0000,  7.0000, -0.0000, -1.0000,  3.0000, -5.0000, -4.0000],
        [-2.0000, -7.0000, -6.0000,  1.0000,  4.0000,  2.0000,  1.0000,  6.0000,
          1.0000, -0.0000, -7.0000,  0.0000, -0.0000, -3.0000, -4.0000,  7.0000],
        [ 2.0000,  1.0000, -1.0000, -0.0000,  4.0000, -1.0000,  0.0000, -1.0000,
         -0.0000, -3.0000,  2.0000,  0.0000,  0.0000,  1.0000,  0.0000,  3.0000],
        [-3.0000,  7.0000,  7.0000, -1.0000, -2.0000,  7.0000,  1.0000,  7.0000,
          3.0000,  7.0000,  1.0000, -0.0000, -0.0000, -7.0000,  7.0000, -3.0000],
        [ 5.0000, -4.0000,  7.0000,  0.0000,  4.0000,  2.0000,  3.0000, -3.0000,
          7.0000,  5.0000,  3.0000,  0.0000,  1.0000, -3.0000, -2.0000,  6.0000],
        [ 3.0000, -3.0

In [26]:
### Complete this cell ###
ic_tile_id = 0 
oc_tile_id = 0 
#print(psum.size())

for kij in range(psum.size(4)):
    psum_tile = psum[ic_tile_id,oc_tile_id,:,:,kij]
    bit_precision = 16
    file = open('psum'+str(kij)+'.txt', 'w') #write to file
    file.write('#time0col17[msb-lsb],time0col16[msb-lst],....,time0col0[msb-lst]#\n')
    file.write('#time1col17[msb-lsb],time1col16[msb-lst],....,time1col0[msb-lst]#\n')
    file.write('#................#\n')

    for i in range(psum_tile.size(1)):  # time step
        for j in range(psum_tile.size(0)): # row #
            if round(psum_tile[7-j,i].item())<0:
                psum_bin = '{0:04X}'.format(round(psum_tile[7-j,i].item()+2**bit_precision))
            else:
                psum_bin = '{0:04X}'.format(round(psum_tile[7-j,i].item()))
            for k in range(4):
                file.write(psum_bin[k])        
            file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file  

#psum_tile[:,:]

In [31]:
bit_precision = 16
file = open('out.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')
# relu
out = torch.relu(torch.reshape(output_int[0,:,:,:], (output_int[0,:,:,:].size(0), -1)))
print(out.size())
for i in range(out.size(1)):  # nijg #
    for j in range(out.size(0)): # row #
        temp=round(out[7-j,i].item())
        if(temp < 0 ):
            temp=temp+2**bit_precision
        W_bin = '{0:016b}'.format(temp)
        for b in range(bit_precision):
            file.write(W_bin[b])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file  

torch.Size([8, 16])
